 # Instalando el ambiente

1. Create un nuevo virtualenv, activalo y actualizá `pip`: `python -m venv venv && source venv/bin/activate.fish && python -m pip install --upgrade pip setuptools`,
2. Instalá *Jupyter Lab*: `python -m pip install jupyterlab`,
3. Instalá las \"Google libs\": `python -m pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib`,
4. Creá un link simbólico que se llame `credentials.json` al JSON que tiene tus credenciales bajadas,

Se crea con el objetivo de guardar todas las librerias y dependencias necesarias que requiera el proyecto. 

# Etapa 1 Extración de los datos de los pdf
En este Jupyter notebook creo los bloques con todos los ejemplos para recopilar los direrentes tipos de datos en los pdf.
A continuación menciono que no mer sirvio y porque lo descarte. 


### 1) No me sirvio utilizar PyPDF2
Es complicado de entender la información ya que me trae metadatos como la fecha de creación del archivo, autor y demás, por lo anterior concluí no me sirve para mi desarrollo. 
Intenté con otro código pero me sigue generando un error ya teniendo instalado el paquete de PdfReader

In [2]:
import PyPDF2 
import PdfReader
pdf_file_obj= open("D:\\Jazmin\\Archivos Personales\\Facultad\\Facultad Analista de Sistemas\\5 CUATRI\\4Practica Profesional\\Recibos\\ARG\\PruebaAR02_IVA_DDJJ.pdf", "rb")
# pdf_read= PyPDF2.PdfFileReader(pdf_file_obj)
# info =pdf_read.getDocumentInfo()
# print(info)
# print(pdf_read.pages)
# print(pdf_read.pages[0].extract_text())

reader = PdfReader(pdf_file_obj)
text = reader.pages[0].extract_text()
assert "Total" in text, text

ModuleNotFoundError: No module named 'PyPDF2'

In [ ]:
from PyPDF2 import PdfFileReader, PdfFileWriter,PdfFileMerger
pdf_file_obj= open("D:\\Jazmin\\Archivos Personales\\Facultad\\Facultad Analista de Sistemas\\5 CUATRI\\4Practica Profesional\\Recibos\\ARG\\PruebaAR02_IVA_DDJJ.pdf", "rb")
template = PdfFileReader(pdf_file_obj)
output = PdfFileWriter()

for i in range(template.getNumPages()):
   page = template.getPage(i)
   print(page)
   page.mergePage(watermark.getPage(0))
   output.addPage(page)
# file = open('waterMarked_PDF.pdf', 'wb')
# output.write(file)

: 

### 2) No me sirvio utilizar fitz
Intento con html pero no tiene un buen formato para extraer información. 

In [ ]:
import fitz 
doct=fitz.open("D:\\Jazmin\\Archivos Personales\\Facultad\\Facultad Analista de Sistemas\\5 CUATRI\\4Practica Profesional\\Recibos\\COL\\VIMEO_SD_PRESENTADA_1BIM_2022.pdf")
pag=doct.load_page(0)
resultado=pag.search_for("Transactions ")
extraer = pag.get_textbox(resultado)
pag.get_text("xhtml")
for line in convertido:
    # print(line)
# print(convertido)
doct_html=open("documento.html","w")
doct_html.write(convertido)
doct_html.close()
print(doct.get_toc())

: 

### 3) Que necesito y utilizo?

 * Leer los formularios para ello debo instalar "tabula" y "tabulate" para leer las tablas de los pdfs ya que los datos estan separados por tablas.
 * También necesito instalar Pandas para analizar los datos y adicional lo requiere Tabula.
 * Numpy sirve para analizar números.(aún no se ha utilizado) 

#### 3.1 Lectura de formatos

In [ ]:
from tabula import read_pdf
from tabulate import tabulate
import pandas as pd
import numpy as np
import re #  Es para la notación de cadena raw de Python para los patrones de Unicode (str)
import locale # separador de mil en US O UK

#**Formulario IvA-ARG
ruta_arg = "D:\\Jazmin\\Archivos Personales\\Facultad\\Facultad Analista de Sistemas\\5 CUATRI\\4Practica Profesional\\Recibos\\ARG\\PruebaAR02_IVA_DDJJ.pdf"
ruta_col = "D:\\Jazmin\\Archivos Personales\\Facultad\\Facultad Analista de Sistemas\\5 CUATRI\\4Practica Profesional\\Recibos\\COL\\Decla. IVA II BIM 2022 - CO10.PDF"
f_Arg = read_pdf(ruta_arg,multiple_tables=True)
f_Col_300 = read_pdf(ruta_col,multiple_tables=True)

##### 3.1.1 Lectura de todo el pdf Argentina, extracción y captura de cada item. 

In [ ]:
año_p= f_Arg[0].iloc[[1],[3]] # pendiente separar cuit y periodo fiscal 
año_p,len(año_p),type(año_p),año_p.values[0][0] # Identifico como recorrer la lineda y se accede por medio de una array
año_pf=año_p.values[0][0]
n=año_pf.split(" ")[0]
period= str(n.split("-")[0]) #ok periodo
año= int(n.split("-")[1]) #ok año


form_unido=f_Arg[0].iloc[[4],[0]] #pendiente separar formulario y razon social 
a= form_unido.values[0][0]
n_for = a.split(" ")[0] # ok formulario

razon_soc2= a.split(" ")[7]
razon_soc1= a.split(" ")[6]+" "+razon_soc2 # ok razon social


verifi= f_Arg[0].iloc[[3],[3]] #pendiente separar form, CUIT, actividad principal y N° verificador
cod_verif= verifi.values[0][0].split(" ")[1] # lo identifico, separo y accedo

#Lectura accede a la primera y segunda sección 
for fila in f_Arg[0].loc[11:].itertuples():#salo las primeras 11 filas
    print(fila[1], fila[4])



##### a) Como extraer CUIT de los pdf? (Argentina)
Debo importar "re" para buscar expresiones regulares

In [ ]:
reg = re.compile(r'[0-9]{2}-[0-9]{8}-[0-9]{1}') # Creo el expresion regular para encontrar el CUIT
id_RazonSocial = reg.search(str(f_Arg[0].iloc[[3],[3]])) # Acá eligo la celda donde se encuentra el CUIT junto con otro texto
print(id_RazonSocial.group(0)) # Imprimo el resultado CUIT

##### b) Como convertir un str a float? 
Debo de importar libreria "locale"

In [ ]:
#Saldo técnico a favor de AFIP
saldo_AFIP = f_Arg[0].iloc[[43],[3]] #Saldo de impuesto a favor de AFIP
sal_entid = saldo_AFIP.values[0][0].replace("." , "") #Accedo al valor pero tengo que limpiar datos para volverlo float 
#locale.setlocale(locale.LC_ALL, 'en_US.UTF-8') solo para . y ,  pero en entero
locale.setlocale(locale.LC_ALL, 'nl_NL') #valor str convertido a float 
valuo_gob= locale.atof(sal_entid)
print(type(valuo_gob), valuo_gob ) # Pendiente convertirlo en numero

#--------------------------------------
#Saldo técnico a favor del responsable
saldo_respon = f_Arg[0].iloc[[42],[3]] #Saldo de impuesto a favor de AFIP
sal_respon = saldo_respon.values[0][0]  #Accedo al valor pero tengo que limpiar datos para volverlo float 
locale.setlocale(locale.LC_ALL, 'nl_NL') #valor str convertido a float 
valuo_respon= locale.atof(sal_respon)
print(type(valuo_respon), valuo_respon ) # Pendiente convertirlo en numero


##### c) Fomulario de Colombia con dos divisiones 
Debo de importar libreria de pandas 
from seaborn "import load_dataset"

In [ ]:
#Lectura accede a la primera y segunda sección 
for fila in f_Col_300[0].itertuples():#salo las primeras 11 filas
    print(fila)
#print(f_Col_300)



In [ ]:
razon_soc1 = f_Col_300[0].iloc[9,[0]].values # linea 9 col 1

period= f_Col_300[0].iloc[3,[5]].values[0]
reg_per = re.compile(r'Período [1-6]{1}')
perio_f= reg_per.search(period)
valor = perio_f.group(0).split(' ')
valor_periodo = valor[1] # valor periodo asegurado
cod_verif = f_Col_300[0].iloc[4,[7]].values 
año =  f_Col_300[0].iloc[3,[0,1,2,3]].values  # linea verlo de otra forma sacar el año tiene str y  int
saldo_pagar= f_Col_300[0].iloc[52,[7]]
saldo_favor= f_Col_300[0].iloc[54,[8]]

print(valor_periodo, razon_soc1, año, saldo_pagar)
print(saldo_favor)




### 4) Libreria pdf pluber ##


En el proceso de revisión y ejecución de extracción de los datos elegí extraerlos con la libreria pdf plumber, es mucho mas legible el proceso de los datos.

Docuemntación
[https://github.com/jsvine/pdfplumber#extracting-form-values]


In [ ]:
import pdfplumber
import re
ruta_col = "D:\\Jazmin\\Archivos Personales\\Facultad\\Facultad Analista de Sistemas\\5 CUATRI\\4Practica Profesional\\Recibos\\COL\\Decla. IVA II BIM 2022 - CO10.PDF"

with pdfplumber.open(ruta_col) as pdf:
    pag = pdf.pages[0]
    text = pag.extract_text()
form= text.split('\n')[0]
form_n= text.split('\n')[3]
nombre = text.split('\n')[4]

   

# print( form, form_n)
# print(text)
#Renglon  y valor
# for m in re.finditer(r'[0-9]{2}\s(\d{1,3}[,]?)+', text):
#     print(m.group(0))

# for m in re.finditer(r'[0-9]{2}\s(\d{1,3}[,]?)+', text):
#     print(m.group(0))



# Lectura de formulario descripción, renglon y valor
for m in re.finditer(r'.+?\s\d{2}\s(\d{1,3}[,]?){1,}', text):
    print(m.group(0))

In [ ]:

# Obtener y limpiar datos e una linea
#a = re.compile(r'1. Año\s([0-9]\s){4}') 
#b = a.match(text)
b= re.search(r'Año\s([0-9]\s){4}', text)
año = re.search(r'([0-9]\s){4}',b.group(0)) 
periodo = re.search(r'([1-6]\s){1}',b.group(0)) 
print(año.group(0).replace(" ", ""), periodo.group(0).replace(" ", ""))



In [ ]:
from decimal import *

Context(2,2,)

print(Decimal("666,868,000", context={ }))

En el formato de de Mexico no leia correctamente el nombre y unia todas las palabras, se tuvo que revisar la libreria de pdfplumber


#.extract_text(x_tolerance=3, y_tolerance=3, layout=False, x_density=7.25, y_density=13, **kwargs)

Cuando layout=False: Agrega espacios donde la diferencia entre el x1 de un carácter y el x0 del siguiente es mayor que [x_tolerance]. Agrega caracteres de nueva línea donde la diferencia entre el doctop de un carácter y el doctop del siguiente es mayor que y_tolerance.

Referencia:

https://github.com/jsvine/pdfplumber#python-library


In [ ]:
from posixpath import split
import pdfplumber
import re
#  Es para la notación de cadena raw de Python para los patrones de Unicode (str)
import locale # separador de mil en US O UK


ruta_mex = "./../../../app/dataextraction/Recibos/MEX/MEX_VAT_Feb2022_Detail.pdf"

cconcepto= []
with pdfplumber.open(ruta_mex) as pdf:
    pag = pdf.pages[0]
    
    text = pag.extract_text()
    lineas =text.split('\n')

concepto ={}
print(text)
for linea in lineas:
    concepto.append(linea)

# Etapa 2 Creación Entorno Docker (ver together love)

Se crea configuraciones y ambiente para que se ejecute Docker se incorpora Django para que funcione aplicación. 
El proceso que se tuvo que realizar es el siguiente:

sin embargo se tuvo que incluir propias especificas de python.
1. Se agrega un [Dockerfile](https://docs.docker.com/engine/reference/builder/) para ejecutar el
aplicación con [Docker](https://docs.docker.com/get-started/).

2. Antes de ejecutar su aplicación con un contenedor acoplable, debe crear su imagen acoplable.
3. Descargue una imagen oficial de python y ingreso a  Docker 
4. La imagen no tiene soporte de requerimientos por eso incluyo adicional unas librerias propias de lo que voy a utilizar [Django] (https://docs.djangoproject.com/en/4.1/).Es por eso que hay un archivo `requirements.txt`.
5. El archivo `requirements.txt` se exportó con `poetry`.

## a) Que configiraciones adicionales tuve que realizar para mi base de datos?

1. Tuve que crear un Mock para simular la realidad, se incluyen unos parametros para que pueda leer los archivos. [Test_commands.py]

2. Se llama la funcion y la pasa [database = defaul] me crear una base al iniciar, y verfica que se llamo con  [assert_called.with]

3. Se crear archivo [wait_for_db.py] comando que verifica si esta **conectada** se crea una bandera [banderas_db.up]
Mientras que sea falso, salta la sentencia True generando directamente un **except** . Esta prueba me identifica que esta funcionando con un loop, sino conecta salta el error hasta que pueda conectar con la base de datos. 

4. Me genero un *error* "For production one .. paycocp2 advisc you distribution". Lo cual no me permitia visualizar si estaba corriendo bien la aplicacion. Probe varias cosas lo cual dejo como enseñanza:

    4.1 El comando **delta** se ingresa a la terminal para visualizar los requirimientos y agregarlos en el archivo. Comando [pp intall-r requerimientos.txt]

    4.2 Se tubo que ingresar libreias y cada vez que se incluia una debia actualizar la imagen [docker compose build].

    4.3 Cuando hace falta una configiracion en la base de datos se salta el error en el test porque no lo reconoce, me fallaba varias veces por que fataba por incluir el tample o los datos fuente de conexión DATABASE.  
    
    4.4 Importante, se deben agregar todas las carpetas para que los pueda encontrar "user" y "core" que son las aplicaciones propias que se agregan para que funcione la aplicaccion. En mi caso tuve que incluir  en el archivo requisitos se debe ingresar en la configuración mi APP Setting InstalV_APP. 

5. Se incluye los test donde comprueba errores mientras que se carga la aplicación, si da exito ejecuta el proceso de la APP si es FALSO, muestra como se deberia tratar tal error. 
    
    5.1 Creación de pruebas unitarias para los modulos de un [usuario] en [django]. El usuario registrarse con su emial y contraseña, el sistema internamente lo crea con **ID**.

    5.2 Prueba united donde se normaliza el email.

    5.3 Cuando se crea un usuario sin email retorna error. 


    